In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sn

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

import warnings
warnings.filterwarnings('ignore')

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('../input/train.csv', parse_dates=["release_date"])
#df = df.dropna()
df.shape
test = pd.read_csv('../input/test.csv', parse_dates=["release_date"])

In [ ]:
df.corr()

In [ ]:
#sn.heatmap(df.corr(),cmap = 'RdYlBu',annot=True)

In [ ]:
df.columns

In [ ]:

df['year'] = df['release_date'].dt.year
#data['day'] = data['release_date'].dt.day
df['month'] = df['release_date'].dt.month
df['release_dayofweek'] = df['release_date'].dt.dayofweek
df['release_quarter'] = df['release_date'].dt.quarter
df['belongs_to_collection'] = df['belongs_to_collection'].apply(lambda x: isinstance(x, str))
df['tagline'] = df['tagline'].apply(lambda x: isinstance(x, str))
df['homepage'] = df['homepage'].apply(lambda x: isinstance(x, str))
df['runtime'].fillna(df['runtime'].mean(), inplace=True)
df.loc[df['year'] > 2018, 'year'] = df.loc[df['year'] > 2018, 'year'].apply(lambda x: x - 100)
df['_budget_year_ratio'] = df['budget']/(df['year']*df['year'])
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

y_val = df.revenue
Y= y_val.apply(np.log10)
feat_movie  = ['_budget_year_ratio','popularity','runtime']
feat = df[feat_movie]
feat.runtime = pd.get_dummies(feat.runtime)
print(feat.isnull().sum())
X_train, X_test, Y_train, Y_test = train_test_split(feat,Y, test_size=0.3,random_state=4)

In [ ]:
params = {'objective':'regression',
          'num_leaves' : 40,
          'min_data_in_leaf' : 10,
          'max_depth' : 6,
          'learning_rate': 0.001,
          "metric": 'rmse',
          "random_state" : 42,
          "lambda_l2" : 0.005,
          "verbosity": -1}

In [ ]:
test = pd.read_csv('../input/test.csv', parse_dates=["release_date"])

test['year'] = test['release_date'].dt.year
test['month'] = test['release_date'].dt.month
test['release_dayofweek'] = test['release_date'].dt.dayofweek
test['release_quarter'] = test['release_date'].dt.quarter
test['year'].fillna(test['year'].mean(), inplace=True)
test['month'].fillna(test['month'].mean(), inplace=True)
test['release_dayofweek'].fillna(test['release_dayofweek'].mean(), inplace=True)
test['release_quarter'].fillna(test['release_quarter'].mean(), inplace=True)
test['belongs_to_collection'] = test['belongs_to_collection'].apply(lambda x: isinstance(x, str))
test['tagline'] = test['tagline'].apply(lambda x: isinstance(x, str))
test['homepage'] = test['homepage'].apply(lambda x: isinstance(x, str))
test['runtime'].fillna(test['runtime'].mean(), inplace=True)
test.loc[test['year'] > 2018, 'year'] = test.loc[test['year'] > 2018, 'year'].apply(lambda x: x - 100)
test['_budget_year_ratio'] = test['budget']/(test['year']*test['year'])

X_t = test[feat_movie]
X_t.shape

In [ ]:
import lightgbm as lgbm

lgbm_train = lgbm.Dataset(X_train,Y_train)
lgbm_eval = lgbm.Dataset(X_test,Y_test,reference = lgbm_train)
print(X_train.shape)
print(X_t.shape)

In [ ]:
print("TRAINING: ")

gbm = lgbm.train(params,lgbm_train,num_boost_round=3000,valid_sets = lgbm_eval,early_stopping_rounds=15)

In [ ]:
X_t.head()

In [ ]:

y_predictions = gbm.predict(X_t)
y_predictions = (10 ** y_predictions)
print("\n",y_predictions)
print(y_predictions.shape)

In [ ]:
#FOR LGBM
submissions= pd.DataFrame({'id':test.id,'revenue':y_predictions},
                         columns = ['id','revenue'])

submissions.to_csv('submission.csv',index=False)
